# Chapter 11

## 11.1
根据优先级规则，3+4 -> 5 和3 -> 4+5是如何被求值的?

In [1]:
3+4 -> 5

res0: (Int, Int) = (7, 5)

In [1]:
3 -> 4+5

cmd1.sc:1: type mismatch;
 found   : Int(5)
 required: String
val res1 = 3 -> 4+5
                  ^

: 

A: -> 和 + 优先级是一样的，所以从左到右运算

## 11.2
BitInt类有一个pow方法，但没有用操作符字符。Scala类库的设计者为什么没有选用**（像Fortran那样）或者^（像Pascal那样）作为乘方操作符呢？

A: 因为优先级问题，在scala中*优先于^，但数学中乘方优先于乘法。所以没有提供^作为乘方的操作符。

## 11.3
实现Fraction类，支持 + - * / 操作。支持约分，例如将15/-6变成-5/2。除以最大公约数，像这样：
```scala
class Fraction(n: Int, d: Int) {
  private val num: Int = if (d == 0) 1 else n * sign(d) /gcd(n, d);
  private val den: Int = if (d == 0) 0 else d * sign(d) /gcd(n, d);
  override def toString = num + "/" + den
  def sign(a: Int) = if (a > 0) 1 else if (a < 0) -1 else 0
  def gcd(a: Int, b: Int): Int = if (b == 0) abs(a) else gcd(b, a % b)
  ...
}
```

In [2]:
import scala.math.abs

class Fraction(n: Int, d: Int) {
    private val num: Int = if (d == 0) 1 else n * sign(d) / gcd(n, d)
    private val den: Int = if (d == 0) 0 else d * sign(d) / gcd(n, d)
    override def toString = num + "/" + den
    def sign(a: Int) = if (a > 0) 1 else if (a < 0) -1 else 0
    def gcd(a: Int, b: Int): Int = if (b == 0) abs(a) else gcd(b, a % b)
    
    def +(that: Fraction): Fraction = {
        Fraction(this.num * that.den + that.num * this.den, this.den * that.den)
    }
    
    def -(that: Fraction): Fraction = {
        Fraction(this.num * that.den - that.num * this.den, this.den * that.den)
    }
    
    def *(that: Fraction): Fraction = {
        Fraction(this.num * that.num, this.den * that.den)
    }
    
    def /(that: Fraction): Fraction = {
        Fraction(this.num * that.den, this.den * that.num)
    }
}

object Fraction {
    def apply(n: Int, d: Int) = new Fraction(n, d)
}

val result = new Fraction(2, 3) + new Fraction(4, 5)

import scala.math.abs


defined class Fraction
defined object Fraction
result: Fraction = 22/15

## 11.4
实现一个Money类，加入美元和美分字段。提供+、-操作符以及比较操作符==和<。举例来说Money(1, 75) + Money(0, 50) == Money(2, 25)应为true。你应该同时提供*和/操作符吗？为什么？
A: 金额乘除没有意义

In [5]:
class Money (val dollar: Int, val cent: Int) {
    private def toCent = this.dollar * 100 + this.cent
    
    def +(that: Money) {
        Money(this.dollar + that.dollar, this.cent + that.cent)
    }
    
    def -(that: Money) {
        Money(0, this.toCent - that.toCent)
    }
    
    def ==(that: Money): Boolean = {
        this.dollar * 100 + this.cent == that.dollar * 100 + that.cent
    }
    
    def <(that: Money): Boolean = {
        this.dollar * 100 + this.cent < that.dollar * 100 + that.cent
    }
    
    override def toString = { "dollar = " + this.dollar + " cent = " + this.cent}
}

object Money {
    def apply(dollar: Int, cent: Int) = {
        val d = dollar + cent / 100
        new Money(d, cent % 100)
    }
}

Money(2, 50)
println(Money(10, 10) < Money(5, 600))

true


defined class Money
defined object Money
res4_2: wrapper.wrapper.Money = dollar = 2 cent = 50

## 11.5
提供操作符用于构造HTML表格。例如：
```
Table() | "Java" | "Scala" || "Gosling" | "Odersky" || "JVM" | "JVM,.NET"
```
应产出:
```
<table><tr><td>Java</td></tr><td>Scala</td></tr><tr><td>Gosling…
```

In [2]:
class Table {
    var s: String = ""
    
    def |(str: String): Table = {
        Table(this.s + "<td>" + str + "</td>")
    }
    
    def ||(str: String): Table = {
        Table(this.s + "</tr><tr><td>" + str + "</td>")

    }
    
    override def toString: String = {
        "<table><tr>" + this.s + "</tr></table>"
    }
}

object Table {
    def apply(): Table = {
        new Table
    }
    
    def apply(str: String): Table = {
        val t = new Table
        t.s = str
        t
    }
}

Table() | "Java" | "Scala" || "Gosling" | "Odersky" || "JVM" | "JVM,.NET"

defined class Table
defined object Table
res1_2: wrapper.wrapper.Table = <table><tr><td>Java</td><td>Scala</td></tr><tr><td>Gosling</td><td>Odersky</td></tr><tr><td>JVM</td><td>JVM,.NET</td></tr></table>

## 10.6
提供一个ASCIIArt类，其对象包含类似这样的图形:
```
     /\_/\
	( ' ' )
    (  -  )
     | | |
    (__|__)
```
提供将两个ASCIIArt图形横向或纵向结合的操作符。选用适当优先级的操作符命名。纵向结合的示例
```
     /\_/\     -----
    ( ' ' )  / Hello \
    (  -  ) <  Scala |
     | | |   \ Coder /
    (__|__)    -----
```

In [4]:
import scala.collection.mutable.ArrayBuffer

class ASCIIArt(str: String) {
    val arr: ArrayBuffer[ArrayBuffer[String]] = new ArrayBuffer[ArrayBuffer[String]]()

    if (str != null && !str.trim.equals("")) {
        str.split("[\r\n]+").foreach(
            line => {
                val s = new ArrayBuffer[String]()
                s += line
                arr += s
            }
        )
    }

    def this() {
        this("")
    }

    /** * 横向结合 * @param other * @return */
    def +(other: ASCIIArt): ASCIIArt = {
        val art = new ASCIIArt()
        // 获取最大行数
        val length = if (this.arr.length >= other.arr.length) this.arr.length else other.arr.length
        for (i <- 0 until length) {
            val s = new ArrayBuffer[String]()
            // 获取this中的行数据, 行数不足,返回空行
            val thisArr: ArrayBuffer[String] = if (i < this.arr.length) this.arr(i) else new ArrayBuffer[String]()
            // 获取other中的行数据, 行数不足,返回空行
            val otherArr: ArrayBuffer[String] = if (i < other.arr.length) other.arr(i) else new ArrayBuffer[String]()
            // 连接this
            thisArr.foreach(s += _)
            // 连接other
            otherArr.foreach(s += _)
            art.arr += s
        }
        art
    }

    /** * 纵向结合 * @param other * @return */
    def *(other: ASCIIArt): ASCIIArt = {
        val art = new ASCIIArt()
        this.arr.foreach(art.arr += _)
        other.arr.foreach(art.arr += _)
        art
    }

    override def toString = {
        var ss: String = ""
        arr.foreach(ss += _.mkString(" ") + "\n")
        ss
    }
}

val a = new ASCIIArt(
        """ /\_/\ |( ' ' ) |( - ) | | | | |(__|__) | """.stripMargin)
val b = new ASCIIArt(
        """ ----- | / Hello \ |< Scala | | \ Coder / | ----- | """.stripMargin)

println(a + b)
println(a * b)

 /\_/\ |( ' ' ) |( - ) | | | | |(__|__) |   ----- | / Hello \ |< Scala | | \ Coder / | ----- | 

 /\_/\ |( ' ' ) |( - ) | | | | |(__|__) | 
 ----- | / Hello \ |< Scala | | \ Coder / | ----- | 



import scala.collection.mutable.ArrayBuffer


defined class ASCIIArt
a: wrapper.wrapper.ASCIIArt =  /\_/\ |( ' ' ) |( - ) | | | | |(__|__) | 

b: wrapper.wrapper.ASCIIArt =  ----- | / Hello \ |< Scala | | \ Coder / | ----- | 


## 11.7
实现一个BigSequence类,将64个bit的序列打包在一个Long值中。提供apply和update操作来获取和设置某个具体的bit

In [5]:
class BigSequence(private var value: Long = 0) {
    def update(bit: Int, state: Int) = { 
        if (state == 1) 
            value |= (state & 1L) << bit % 64 
        else 
            value &= ~(1L << bit % 64) } 
    
    def apply(bit: Int): Int = if (((value >> bit % 64) & 1L) > 0) 1 else 0 
    
    override def toString = "%64s".format(value.toBinaryString).replace(" ", "0") 
}

val x = new BigSequence() 
x(5) = 1 
x(63) = 1 
x(64) = 1 

println(x(5))
println(x)

1
1000000000000000000000000000000000000000000000000000000000100001


defined class BigSequence
x: BigSequence = 1000000000000000000000000000000000000000000000000000000000100001

## 10.8
提供一个Matrix类——你可以选择需要的是一个2x2的矩阵，任意大小的正方形矩阵，或是m x n的矩阵。支持+和*操作。*操作应同样适用于单值，例如mat*2。单个元素可以通过mat(row, col)得到。 

In [12]:
class Matrix(val m: Int, val n: Int) {
    private val value = Array.ofDim[Double](m, n)
    
    def update(row: Int, col: Int, v: Double) = value(row)(col) = v
    
    def apply(row: Int, col: Int): Double = value(row)(col)
    
    def +(that: Matrix): Matrix = {
        require(n == that.n)
        require(m == that.m)
        
        val res = new Matrix(m, n)
        for (i <- 0 until m; j <- 0 until n) {
            res(i, j) = this(i, j) + that(i, j)
        }
        res
    }
    
    def -(that: Matrix): Matrix = {
        this + that * (-1)
    }
    
    def *(that: Matrix): Matrix = {
        require(n == that.m)
        
        val res = new Matrix(m, that.n)
        for (i <- 0 until m; j <- 0 until that.n) {
            res(i, j) = (for (k <- 0 until n) yield value(i)(k) * that.value(j)(k)).sum
        }
        res
    }
    
    def *(factor: Double): Matrix = {
        val res = new Matrix(m, n)
        for (i <- 0 until m; j <- 0 until n) {
            res(i, j) = this(i, j) * factor
        }
        res
    }
    
    override def toString = value.map(_.mkString(" ")).mkString("\n")
}

val x = new Matrix(2, 2)
x(0, 0) = 1
x(0, 1) = 2
x(1, 0) = 3
x(1, 1) = 4

println(x)
println()
println(x * 2)
println()
println(x * 2 - x)
println()
println((x * 2) * (x * 3))

1.0 2.0
3.0 4.0

2.0 4.0
6.0 8.0

1.0 2.0
3.0 4.0

30.0 66.0
66.0 150.0


defined class Matrix
x: wrapper.wrapper.Matrix = 1.0 2.0
3.0 4.0

## 11.9
为RichFile类定义unapply操作，提取文件路径、名称和扩展名。举例来说，文件/home/cay/readme.txt的路径为/home/cay，名称为readme，扩展名为txt。

In [13]:
class RichFile(val path: String) {}

object RichFile {
    def apply(path: String): RichFile = {
        new RichFile(path)
    }
    
    def unapply(rf: RichFile): Option[(String, String, String)] = {
        if (rf.path == null) {
            None
        } else {
            val reg = """([/\w+]+)/(\w+)\.(\w+)""".r
            val reg(r1, r2, r3) = rf.path
            Some((r1, r2, r3))
        }
    }
}

val rf = RichFile("/home/cay/readme.txt")
val RichFile(r1, r2, r3) = rf
println(r1 + "  " + r2 + "  " + r3)

/home/cay  readme  txt


defined class RichFile
defined object RichFile
rf: RichFile = $sess.cmd12Wrapper$Helper$RichFile@b3f5f49
r1: String = "/home/cay"
r2: String = "readme"
r3: String = "txt"

## 11.10
为RichFile类定义一个unapplySeq，提取所有路径段。举例来说，对于/home/cay/readme.txt，你应该产出三个路径段的序列：home、cay和readme.txt。

In [15]:
class RichFile(val path: String) {}

object RichFile {
    def apply(path: String): RichFile = {
        new RichFile(path)
    }
    
    def unapplySeq(rf: RichFile): Option[Seq[String]] = {
        if (rf.path == null) {
            None
        } else {
            if (rf.path.startsWith("/"))
                Some(rf.path.substring(1).split("/"))
            else
                Some(rf.path.split("/"))
        }
    }
}

val rf = RichFile("/home/cay/readme.txt")
val RichFile(r @ _*) = rf
println(r)

WrappedArray(home, cay, readme.txt)


defined class RichFile
defined object RichFile
rf: wrapper.wrapper.RichFile = $sess.cmd14Wrapper$Helper$RichFile@26aac627
r: Seq[String] = Array("home", "cay", "readme.txt")